# import

In [2]:
import numpy as np
import plotly.graph_objects as go
from scipy.spatial import Delaunay

In [3]:


def create_drop_points(center=(0,0,0), height=0.05, points=100):
    t1 = np.linspace(0, np.pi, points//2)
    x1 = 0.05 * np.cos(t1)
    y1 = 0.05 * np.sin(t1)
    z1 = np.zeros_like(t1)

    t2 = np.linspace(0.1, 1, points//2)
    x2 = 0.05 * np.log(t2)
    y2 = 0.03 * t2
    z2 = 0.02 * t2

    x = np.concatenate([x1, x2]) + center[0]
    y = np.concatenate([y1, y2]) + center[1]
    z = np.concatenate([z1, z2]) + center[2]

    points2d = np.vstack([x, y]).T
    tri = Delaunay(points2d)

    return x, y, z, tri.simplices

def plot_animated_filled_drops(rows=3, cols=7, spacing=1.0, frames_num=3):
    fig = go.Figure()
    colors = ['blue', 'deepskyblue', 'dodgerblue']
    base_z = [0, 0.02, -0.02]

    
    for row in range(rows):
        for col in range(cols):
            x, y, z, simplices = create_drop_points(center=(col*spacing, -row*spacing, base_z[row]))
            fig.add_trace(go.Mesh3d(
                x=x, y=y, z=z,
                i=simplices[:,0], j=simplices[:,1], k=simplices[:,2],
                color=colors[row], opacity=0.9,
                flatshading=True,
                name=f"Drop {row}-{col}"
            ))

   
    frames = []
    for frame_idx in range(frames_num):
        data = []
        for row in range(rows):
            for col in range(cols):
                z_offset = base_z[row] + 0.02 * np.sin(2 * np.pi * frame_idx / frames_num)
                x, y, z, simplices = create_drop_points(center=(col*spacing, -row*spacing, z_offset))
                data.append(go.Mesh3d(
                    x=x, y=y, z=z,
                    i=simplices[:,0], j=simplices[:,1], k=simplices[:,2],
                    color=colors[row], opacity=0.9,
                    flatshading=True,
                ))
       
        cloud_x = np.random.normal(loc=cols * spacing / 2, scale=1.5, size=1000)
        cloud_y = np.random.normal(loc=0.5, scale=0.5, size=1000)
        cloud_z = np.random.normal(loc=0.08, scale=0.01, size=1000)
        data.append(go.Scatter3d(
            x=cloud_x, y=cloud_y, z=cloud_z,
            mode='markers',
            marker=dict(size=4, color='lightblue', opacity=0.5),
            name="Cloud"
        ))

        frames.append(go.Frame(data=data, name=f'frame{frame_idx}'))

    fig.frames = frames

    cloud_x = np.random.normal(loc=cols * spacing / 2, scale=1.5, size=1000)
    cloud_y = np.random.normal(loc=0.5, scale=0.5, size=1000)
    cloud_z = np.random.normal(loc=0.08, scale=0.01, size=1000)
    fig.add_trace(go.Scatter3d(
        x=cloud_x, y=cloud_y, z=cloud_z,
        mode='markers',
        marker=dict(size=4, color='lightblue', opacity=0.5),
        name="Cloud"
    ))

    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[-1, cols*spacing], title='X'),
            yaxis=dict(range=[-rows*spacing, 1], title='Y'),
            zaxis=dict(range=[-0.1, 0.15], title='Z'),
            camera=dict(eye=dict(x=1.5, y=1.5, z=0.6))
        ),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="▶️ Play",
                          method="animate",
                          args=[None, {
                              "frame": {"duration": 500, "redraw": True},
                              "fromcurrent": True,
                              "transition": {"duration": 0},
                              "mode": "immediate",
                              "loop": True
                          }])]
        )]
    )

    fig.show()

plot_animated_filled_drops(rows=3, cols=7, spacing=1)
